In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen 
from urllib.error import URLError
import json
import pandas as pd
import re

In [15]:
def create_soup(link):
    '''Возвращает экземпляр BeautifulSoup для соответсвующей ссылки'''
    page = urlopen(link)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")
    return BeautifulSoup(html, "html.parser")
    

In [18]:
def process_page(vacancies, vac_dict):
    '''
    Обходит все вакансии на странице и добавляет извлеченные 
    данные в соотвествующий список
    '''
    # Создаем регулярное выражение для парсинга данных о ЗП
    regex = re.compile('vacancy-salary-compensation-type-.*')
    # Итерируем по списку вакансий
    for vacancy in vacancies:
        link = vacancy.find('a')['href']
        # Подготавливаем soup
        vac_soup = create_soup(link)
        # Извлекаем местоположение и отрасль
        try:
            js = json.loads("".join(vac_soup.find("script", {"type":"application/ld+json"}).contents))
            vac_dict['location'].append(js['jobLocation']['address']['addressLocality'])
            vac_dict['industry'].append(' '.join(js['industry']))
        except:
            continue
        # Извлекаем ЗП
        try:
            vac_dict['salary'].append(vac_soup.find('span', {'data-qa':regex}).get_text())   
        except:
            vac_dict['salary'].append(None)
        # Извлекаем нываки
        try:
            skills = vac_soup.findAll('div',{'data-qa':'bloko-tag bloko-tag_inline skills-element'})
            skil = [s.get_text() for s in skills]
            vac_dict['skills'].append(' '.join(skil)) 
        except:
            vac_dict['skills'].append(None)
        # Извлекаем опыт
        try:
            vac_dict['experience'].append(vac_soup.find('span',{'data-qa':'vacancy-experience'}).get_text())
        except: 
            vac_dict['experience'].append(None)

In [ ]:
url = 'https://hh.ru/search/vacancy?area=113&search_field=description&only_with_salary=true&text=&hhtmFrom=vacancy_search_list'
vac_dict = {'location':[], 'industry':[], 'experience': [], 'skills':[], 'salary':[]}
pages = 40
# Итерируем по страницам
for page in range(pages):
    page_link = url + '&page=' + str(page)
    try:
        # Подготавливаем soup
        page_soup = create_soup(page_link)
        # Извлекаем все вакансии на странице
        vacancies = page_soup.find_all('div', class_='vacancy-serp-item-body')
        # Передаем полученный список в функцию для сбора данных о каждой вакансии
        process_page(vacancies, vac_dict)
    except:
        print('Произошла ошибка! Пропуск страницы...')

Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...
Произошла ошибка! Пропуск страницы...


In [ ]:
# Преобразовываем словарь в DataFrame
vacancies = pd.DataFrame.from_dict(vac_dict)

In [765]:
# Записываем DataFrame в csv файл
vacancies.to_csv('vacancies.csv')